### Imports

---


In [1]:
!pip install pyspark==3.3.1
! pip install pandas matplotlib

     |████████████████████████████████| 12.2 MB 166 kB/s eta 0:00:01    |███████                         | 2.7 MB 844 kB/s eta 0:00:12�███████████▏                   | 4.6 MB 242 kB/s eta 0:00:32     |███████████████████████         | 8.8 MB 2.1 MB/s eta 0:00:02
     |████████████████████████████████| 11.6 MB 166 kB/s eta 0:00:01    |█▎                              | 481 kB 360 kB/s eta 0:00:31     |█▊                              | 624 kB 360 kB/s eta 0:00:31     |██████▊                         | 2.4 MB 360 kB/s eta 0:00:26     |██████████▏                     | 3.7 MB 7.0 MB/s eta 0:00:02     |█████████████████████▎          | 7.7 MB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 981 kB/s eta 0:00:01    |▉                               | 440 kB 274 kB/s eta 0:01:02     |█████▎                          | 2.9 MB 2.1 MB/s eta 0:00:07     |███████▋                        | 4.1 MB 224 kB/s eta 0:00:59     |█████████████████████▋          | 11.7 MB 1.4 MB/s eta 0:00:

In [2]:
from pyspark.sql import SparkSession

## Exploração Bronze

In [3]:
spark= SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()


23/03/12 11:38:46 WARN Utils: Your hostname, heveraldo-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/03/12 11:38:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/12 11:38:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.json("../../datalake/Bronze/twitter_datascience/")

In [5]:
df.show()

+--------------------+--------------------+--------------------+------------+
|                data|            includes|                meta|extract_date|
+--------------------+--------------------+--------------------+------------+
|[{163431558285725...|{[{2023-03-10T22:...|{1634343373174714...|  2023-03-10|
|[{527816441, 1634...|{[{2012-03-17T20:...|{1634343098334560...|  2023-03-10|
|[{163284280386779...|{[{2023-03-06T20:...|{1634342096197894...|  2023-03-10|
|[{190097582, 1634...|{[{2010-09-13T02:...|{1634341762171916...|  2023-03-10|
|[{190097582, 1634...|{[{2010-09-13T02:...|{1634341348831645...|  2023-03-10|
|[{292711049, 1634...|{[{2011-05-04T03:...|{1634340456581480...|  2023-03-10|
|[{162292495016314...|{[{2023-02-07T11:...|{1634339727632367...|  2023-03-10|
|[{113289077589150...|{[{2019-05-27T06:...|{1634338823566917...|  2023-03-10|
|[{104136724813970...|{[{2018-09-16T16:...|{1634338543353974...|  2023-03-10|
|[{149968503384990...|{[{2022-03-04T09:...|{1634338185097379...|

In [6]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- impression_count: long (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |    |    |-- withheld: struct (nullable = true)
 |    |    |    |-- copyright: boolean (nu

In [7]:
from pyspark.sql import functions as f

In [8]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- impression_count: long (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)
 |    |-- withheld: struct (nullable = true)
 |    |    |-- copyright: boolean (nullable = true)
 |    |    |-- country_codes: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)



In [9]:
df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- impression_count: long (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [10]:
tweet_df = df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [11]:
tweet_df.show()

+-------------------+-------------------+--------------------+-------------------+----------------+----------+-----------+-----------+-------------+--------------------+
|          author_id|    conversation_id|          created_at|                 id|impression_count|like_count|quote_count|reply_count|retweet_count|                text|
+-------------------+-------------------+--------------------+-------------------+----------------+----------+-----------+-----------+-------------+--------------------+
|1634315582857256960|1634343373174714368|2023-03-10T23:59:...|1634343373174714368|               0|         0|          0|          0|            1|RT @KirkDBorne: E...|
|          150030084|1634343357009854464|2023-03-10T23:59:...|1634343357009854464|               0|         0|          0|          0|           76|RT @RosanaFerrero...|
|1356588046616043527|1634343354526838787|2023-03-10T23:59:...|1634343354526838787|               0|         0|          0|          0|            1|RT

In [12]:
user_df = df.select(f.explode("includes.users").alias("users"))\
  .select("users.*")

In [13]:
user_df.show()

+--------------------+-------------------+--------------------+---------------+--------+
|          created_at|                 id|                name|       username|withheld|
+--------------------+-------------------+--------------------+---------------+--------+
|2023-03-10T22:09:...|1634315582857256960|       Abusadeeq ias|  abusadeeq_ias|    null|
|2010-05-30T22:07:...|          150030084|       Andrés A León|   Andres_ALeon|    null|
|2021-02-02T12:59:...|1356588046616043527|AI Bot by uCloudi...|    uCloudifyAI|    null|
|2017-10-22T14:38:...| 922109833637126147|        Gift Coupons|     CourseGift|    null|
|2011-01-14T08:47:...|          238071685|              Jerome|     JeromeFAJi|    null|
|2012-02-08T12:47:...|          486573105|    inKstall Educare|       Inkstall|    null|
|2015-10-05T19:07:...|         3873598893|     Matthew Gellman|     ProjectNav|    null|
|2009-08-09T15:56:...|           64197067|         Bill Brooks|BROOKSWOODWORKS|    null|
|2012-03-17T20:35:...

In [14]:
user_df.coalesce(1).write.mode("overwrite").json('notebook_output/user')
tweet_df.coalesce(1).write.mode("overwrite").json('notebook_output/tweet')

## Exploração Silver


In [16]:
df_tweet = spark.read.json("../../datalake/Silver/twitter_datascience/tweet/")

In [17]:
tweet_conversas = df_tweet.alias("tweet")\
        .groupBy(f.to_date("created_at").alias("created_date"))\
        .agg(
            f.countDistinct("author_id").alias("n_tweets"),
            f.sum("like_count").alias("n_like"),
            f.sum("quote_count").alias("n_quote"),
            f.sum("reply_count").alias("n_reply"),
            f.sum("retweet_count").alias("n_retweet")
        ).withColumn("weekday", f.date_format("created_date", "E"))


In [18]:

tweet_conversas.orderBy("created_date").show()

+------------+--------+------+-------+-------+---------+-------+
|created_date|n_tweets|n_like|n_quote|n_reply|n_retweet|weekday|
+------------+--------+------+-------+-------+---------+-------+
|  2023-03-09|      53|    84|      1|      2|     1123|    Thu|
|  2023-03-10|      72|    29|      0|      1|     4944|    Fri|
|  2023-03-11|      48|   436|     12|      6|     6246|    Sat|
+------------+--------+------+-------+-------+---------+-------+

